# code

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

dataset = '/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/train.csv'

Mounted at /content/drive


In [ ]:
!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 trl==0.8.6 datasets==2.19.0

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 120.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
import os
import torch
from peft import LoraConfig
from trl import SFTTrainer
import huggingface_hub
huggingface_hub.login('token')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
%cd /content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회

/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회


In [ ]:


base_model = "rtzr/ko-gemma-2-9b-it"

dataset = '/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/train.csv'

# 새로운 모델 이름
new_model = "gemma-Aivle-Ko-3.1-8B"

In [ ]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.float16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 66.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          logging,
                          pipeline)
# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
import pandas as pd

query_dataset = pd.read_csv(dataset)
query_dataset['Question'][0], query_dataset['Answer'][0]

('2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?',
 '2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.')

In [ ]:
prompt = []
for index,row in query_dataset.iterrows():
  prompt.append({"text":row['Question']+row['Answer']})

In [ ]:
prompt[:3]

[{'text': '2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.'},
 {'text': '2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성되어 있습니다.'},
 {'text': '기금이 예산과 다른 점은?기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정부가 편성하고 국회에서 심의・의결한 기금운용계획에 의해 운용됩니다.'}]

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_list(prompt)

In [ ]:
### 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto",
    attn_implementation='eager' #gemma
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00002-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
              base_model,
              trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
      "lm_head",
    ],
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=200,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
)

In [ ]:
# 파인튜닝
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
25,1.892500
50,1.886200
75,1.623900
100,1.534200
125,1.514700
150,1.479700
175,1.497900
200,1.564200
225,1.364900
250,1.480800


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

TrainOutput(global_step=2480, training_loss=0.48006368678423667, metrics={'train_runtime': 4667.1356, 'train_samples_per_second': 1.063, 'train_steps_per_second': 0.531, 'total_flos': 2.193609465280512e+16, 'train_loss': 0.48006368678423667, 'epoch': 10.0})

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

RuntimeError: Index put requires the source and destination dtypes match, got Half for the destination and Float for the source.

In [ ]:
!pip install bitsandbytes
!pip install pdfplumber
!pip install -r requirements.txt
!pip install -U langchain-community
!pip install chromadb

!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.4 MB/s eta 0

In [ ]:
from huggingface_hub import login
import os
import re
from tqdm import tqdm
import chromadb
import argparse
import pandas as pd
import numpy as np
import shutil
from collections import Counter
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
# from get_embedding_function import get_embedding_function
from langchain.vectorstores.chroma import Chroma
from langchain_community.document_loaders import PDFPlumberLoader, PyPDFium2Loader


# Load model directly
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)
import torch


In [ ]:
def extract_alphanumeric(input_string: str) -> str:
    # 정규 표현식을 사용하여 숫자와 영문자만 추출
    result = re.findall(r'[a-zA-Z0-9]+', input_string)
    # 리스트를 문자열로 결합하여 반환
    return ''.join(result)

In [ ]:
def load_pdf_file(SOURCE_PATH):
  # file_loader = PDFPlumberLoader(SOURCE_PATH)
  file_loader = PyPDFium2Loader(SOURCE_PATH)

  return file_loader.load()

In [ ]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [ ]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
Sentence_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
def get_embedding_function(chunks):
  # model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
  # model = SentenceTransformer('jhgan/ko-sroberta-multitask')
  embeddings = []
  for chunk in tqdm(chunks):
    embeddings.append(Sentence_model.encode(chunk))
  return np.array(embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def make_prompt(instruction, n_similer):

  PROMPT = f'''당신은 PDF 문서에서 정보를 검색하고 이를 바탕으로 답변하는 AI입니다.
  당신의 역할은 '중앙정부 재정 정보 질의 응답 AI'이고, 사용자의 역할은 '사용자'입니다.
  다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 답변을 제공해야 합니다.
  답변은 무조건 한문장으로 정리하여 답변하며 질문의 대한 대답을 모두 포함해야 합니다.
  검색 결과만을 사용하여 정보를 검색하여 명확하고 간결한 답변을 제공해야 합니다.
  어떤 상황에서도 한문장으로 답변합니다.
  답변에 한국어만을 사용합니다.
  빈드시 답변에 한국어만을 사용합니다.

  - 검색 결과의 내용을 바탕으로 사용자의 질문에 적합한 답변을 작성합니다.
  - 연도를 포함한 정보는 연도별로 요약하여 설명합니다.
  - 답변은 명확하고 간결하게 작성합니다.
  - 검색 결과의 문장을 최대한 그대로 인용합니다.
  - Answer only in Korean, no matter what.
  ### 검색 결과:
  '''

  instruction = instruction

  # 백터DB 쿼리
  query = instruction

  results = collection.query(
      query_embeddings=Sentence_model.encode([query]),
      n_results=n_similer
  )

  # 결과를 문자열로 변환
  results_content = "\n".join([result["chunk"] for result in results['metadatas'][0]])

  messages = [
      {"role": "system", "content": f"{PROMPT}\n\n\n### 질문:{results_content}"},
      {"role": "user", "content": f"{instruction}"}
  ]

  return messages


In [ ]:
# @title Inference
hugging_face_token = "token" # @param {type:"string"}
csv_path = "train.csv" # @param {type:"string"}
llama_model_id = "Nacho-Cola/llama3.1_report_model_0813" # @param {type:"string"}
similar_n = 5 # @param {type:"integer"}
model_temperature = 0.9 # @param {type:"number"}


# Hugging Face에 로그인
login(token=hugging_face_token)


model.eval()

# CSV 파일 로드
df = pd.read_csv(csv_path)

# 고유한 PDF 경로 추출
pdfs = df['Source_path'].unique()

# 질문 리스트 초기화
Questions = []
for ind, pdf in enumerate(pdfs):
    Questions.append({
        'index': 'pdf_' + str(ind),
        'Source': pdf,
        'Question': df.loc[df['Source_path'] == pdf]['Question']
    })

answers = []  # 답변 리스트 초기화

# 각 질문에 대해 처리
for Question in tqdm(Questions):
    documents = load_pdf_file(Question['Source'])  # PDF 파일 로드
    split_docs = split_documents(documents)  # 문서 분할

    meta_dict = []  # 메타데이터 저장 리스트
    page_content = []  # 페이지 내용 저장 리스트
    for doc in split_docs:
        meta_dict.append(doc.metadata)  # 메타데이터 추가
        page_content.append(doc.page_content)  # 페이지 내용 추가

    embeddings = get_embedding_function(page_content)  # 임베딩 생성

    chroma_client = chromadb.Client()  # ChromaDB 클라이언트 초기화
    collection = chroma_client.create_collection(extract_alphanumeric(Question['index']+Question['Source']+'_test8'))  # 컬렉션 생성

    metadatas = []  # 메타데이터 저장 리스트
    ids = []  # ID 저장 리스트

    for i, article in tqdm(enumerate(split_docs)):  # 분할된 문서 반복
        metadatas.append({
            'source': article.metadata['source'],  # 소스 추가
            'page': article.metadata['page'],      # 페이지 추가
            'chunk': article.page_content           # 문서 내용 추가
        })
        ids.append(str(i))  # ID 추가

    embeddings_list = embeddings.tolist()  # 임베딩 리스트로 변환

    # ChromaDB에 데이터 추가
    collection.add(
        embeddings=embeddings_list,
        metadatas=metadatas,
        ids=ids
    )

    prompts = []  # 프롬프트 저장 리스트
    for instruction in Question['Question']:
        prompts.append(make_prompt(instruction, n_similer=similar_n))  # 프롬프트 생성

    for message in tqdm(prompts):  # 프롬프트 반복
        input_ids = tokenizer.apply_chat_template(
            message,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)  # 입력 ID 생성

        input_ids = input_ids.to(torch.long)  # Long 타입으로 변환


        # 모델을 통해 응답 생성
        outputs = model.generate(
            input_ids = input_ids,
            max_new_tokens=128,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=model_temperature,
            top_p=0.9,
            repetition_penalty=1.1,
        )

        print('\n'+message[1]['content'])
        response = outputs[0][input_ids.shape[-1]:]  # 응답 추출
        answer = tokenizer.decode(response, skip_special_tokens=True)
        answers.append(answer)  # 응답 디코딩하여 추가

        print(answer)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")

100%|██████████| 267/267 [00:04<00:00, 63.67it/s]

267it [00:00, 462377.86it/s]

  0%|          | 0/16 [00:06<?, ?it/s]


RuntimeError: Index put requires the source and destination dtypes match, got Half for the destination and Float for the source.

In [ ]:
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())

    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):

    total_precision = 0
    total_recall = 0
    total_f1_score = 0

    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score

    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)

    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }


result = calculate_average_f1_score(df['Answer'], answers)
print(result)

In [ ]:
model.push_to_hub("Nacho-Cola/llama3.1_report_model_0815")
tokenizer.push_to_hub("Nacho-Cola/llama3.1_report_model_0815")

RuntimeError: The weights trying to be saved contained shared tensors [{'model.embed_tokens.weight', 'lm_head.base_layer.weight'}] that are mismatching the transformers base configuration. Try saving using `safe_serialization=False` or remove this tensor sharing.

In [ ]:
print(input_ids.dtype)  # input_ids의 데이터 타입 확인
print(model.device)  # 모델의 디바이스 확인


torch.int64
cuda:0


# GGUF

In [ ]:
%cd /content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회

In [ ]:
from huggingface_hub import snapshot_download

model_id = 'Nacho-Cola/llama3.1_report_model_0813'

snapshot_download(repo_id=model_id, local_dir='llama_pine_tuned', local_dir_use_symlinks=False, revision="main")

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 32104, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 32104 (delta 82), reused 128 (delta 66), pack-reused 31953 (from 1)
Receiving objects: 100% (32104/32104), 57.52 MiB | 14.19 MiB/s, done.
Resolving deltas: 100% (22969/22969), done.


In [ ]:
!cd llama.cpp && GGML_CUDA=1 make

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include -DGGML_CUDA_USE_GRAPHS  -std=c11   -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_S

In [ ]:
!pip install -r llama.cpp/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 25.6 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.3.1+cu121:
      Successfully uni

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py llama_pine_tuned --outfile llama_fine_tuned_8B.gguf --outtype f32

INFO:hf-to-gguf:Loading model: llama_pine_tuned
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float32 --> F32, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float32 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.uint8 --> F32, shape = {29360128}
Traceback (most recent call last):
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 3953, in <module>
    main()
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 3947, in main
    model_instance.write()
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 387, in write
    self.prepare_tensors()
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 1588, in prepare_tensors
    super().prepare_tensors()
  Fil

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# gguf 파일 경로
file_path = "llama_fine_tuned_8B"

# Hugging Face 저장소 이름
repo_name = "Nacho-Cola/llama_fine_tuned_8B_GGUF"

# 파일 업로드
api.upload_file(
    path_or_fileobj=file_path,
    path_in_repo="hkcode-llama-fine-tuned-8B-0813.gguf",  # 저장소 내 파일 경로
    repo_id=repo_name,
    repo_type="model",  # 모델 저장소에 업로드
)

ValueError: Provided path: 'llama_fine_tuned_8B' is not a file on the local file system

# GPT

In [ ]:
!pip install --upgrade transformers
!pip install datasets torch

In [ ]:
import os
import torch
import huggingface_hub
huggingface_hub.login('token')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

dataset = '/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/train.csv'

In [ ]:
import pandas as pd

query_dataset = pd.read_csv(dataset)
query_dataset['Question'][0], query_dataset['Answer'][0]

In [ ]:
prompt = []
for index,row in query_dataset.iterrows():
  prompt.append({"text":row['Question']+row['Answer']})

In [ ]:
prompt[:3]

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_list(prompt)

In [ ]:
from datasets import load_dataset

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)


In [ ]:
tokenized_datasets

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=4
)


In [ ]:
device = torch.device("cpu")
model.to(device)

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

trainer.train()
